In [1]:
from canonical_toolkit.morphology.visual.plots.bokeh_grid_plotter import BokehConfig, BokehGridPlotter

In [ ]:
from bokeh.plotting import output_notebook
import numpy as np
from PIL import Image
from io import BytesIO
import base64
output_notebook()

Loading BokehJS ...

In [3]:
def generate_test_data(n_points=100):
    # 1. Coordinate sets (Scatter Nx2)
    umap = np.random.randn(n_points, 2)
    tsne = np.random.randn(n_points, 2) * 5.0
    
    # 2. Heatmap set (MxN)
    heatmap = np.sin(np.outer(np.linspace(-3, 3, 20), np.linspace(-3, 3, 20)))
    
    # 3. Global IDs
    ids = np.arange(n_points)
    
    # 4. Generate some simple base64 thumbnails
    thumbnails = []
    for i in ids:
        # Create a small color-coded square based on ID
        img = Image.new('RGB', (40, 40), color=(i * 2 % 255, 120, 200))
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        b64 = base64.b64encode(buffered.getvalue()).decode()
        thumbnails.append(f"data:image/png;base64,{b64}")
        
    return umap, tsne, heatmap, ids, thumbnails

umap_data, tsne_data, heatmap_data, robot_ids, robot_thumbs = generate_test_data()

In [4]:
# Setup 1x3 Grid
config = BokehConfig(sidebar_enabled=True, plot_width=300)
plotter = BokehGridPlotter(n_rows=1, n_cols=3, config=config)

# Bulk load images into cache
plotter.add_thumbnails(robot_ids, robot_thumbs)

# Create a 2D grid structure: Row 1 has UMAP, t-SNE, and a Heatmap
data_grid = [[umap_data, tsne_data, heatmap_data]]
id_grid = [[robot_ids, robot_ids, None]] # Heatmap doesn't take IDs
titles = [["UMAP Projection", "t-SNE Projection", "Density Heatmap"]]

plotter.add_2D_numeric_data(data_grid, global_ids_2d=id_grid, titles_2d=titles)

# VECTORIZED TEST: Highlight robots 10 through 20 in Orange
# This should reflect in both UMAP and t-SNE simultaneously
plotter[:, :2].set_color("#ff7f0e", global_ids=np.arange(10, 21))

plotter.show(super_title="Test 1: 2D Ingestion & Linked Styling")

In [5]:
# Setup 2x2 Grid
plotter_2 = BokehGridPlotter(n_rows=2, n_cols=2)
plotter_2.add_thumbnails(robot_ids, robot_thumbs)

# Create 4 variations of the same data
data_list = [umap_data, tsne_data, umap_data * 0.5, tsne_data * 2]
ids_list = [robot_ids] * 4
titles = ["UMAP A", "t-SNE A", "UMAP B (Scaled)", "t-SNE B (Scaled)"]

plotter_2.add_numeric_data(data_list, global_ids_list=ids_list, titles=titles)

# TEST: Set a specific column title
# plotter_2[:, 0].set_title("UMAP Column")

plotter_2.show(super_title="Test 2: Multi-Row Brushing & Gallery")

In [6]:
def generate_thumbnails_for_ids(ids, width: int = 50, height: int = 50) -> list[str]:
    """
    Generates placeholder Base64 thumbnails for testing.
    Each ID gets a unique color based on its value.
    """
    thumbnails = []
    for i in ids:
        # Create a unique color based on ID
        r = (i * 37) % 255
        g = (i * 53) % 255
        b = (i * 97) % 255
        img = Image.new('RGB', (width, height), color=(r, g, b))
        
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        b64 = base64.b64encode(buffered.getvalue()).decode()
        thumbnails.append(f"data:image/png;base64,{b64}")
    return thumbnails

def path_to_base64(path: str) -> str:
    """Utility to convert a local image file to a Bokeh-ready Base64 string."""
    with open(path, "rb") as image_file:
        b64 = base64.b64encode(image_file.read()).decode()
        # Detect extension
        ext = path.split('.')[-1].lower()
        return f"data:image/{ext};base64,{b64}"

In [ ]:
from bokeh.plotting import output_notebook
import numpy as np

output_notebook()

# 1. Setup data
N_SIDE = 10  # 10x10 heatmap
N_SCATTER = 100

# Scatter data
umap_data = np.random.randn(N_SCATTER, 2)
scatter_ids = np.arange(N_SCATTER)

# Heatmap data (10x10)
heatmap_data = np.random.rand(N_SIDE, N_SIDE)

# IMPORTANT: Heatmap IDs must be TUPLES (row_id, col_id) for two images!
heatmap_row_ids = np.arange(N_SIDE)  # IDs 0-9 for rows
# Create tuple pairs: cell (i,j) gets tuple (i, j)
id_pairs = np.array([
    [(heatmap_row_ids[i], heatmap_row_ids[j]) for j in range(N_SIDE)] 
    for i in range(N_SIDE)
], dtype=object)

print(f"id_pairs shape: {id_pairs.shape}")  # Should be (10, 10)
print(f"id_pairs[0,0]: {id_pairs[0,0]}")    # Should be (0, 0)
print(f"id_pairs[2,5]: {id_pairs[2,5]}")    # Should be (2, 5)

# Generate thumbnails for IDs 0-9 (used in heatmap) and 0-99 (used in scatter)
all_ids = np.arange(N_SCATTER)
robot_thumbs = generate_thumbnails_for_ids(all_ids)

# 2. Initialize Plotter
config = BokehConfig(
    plot_width=400, 
    plot_height=400, 
    global_axis=False, 
    hover_img_height=64
)
plotter = BokehGridPlotter(n_rows=1, n_cols=2, config=config)

# 3. Load thumbnail cache (keyed by individual ID, not tuples)
plotter.add_thumbnails(all_ids, robot_thumbs)

# 4. Add Data
# - Scatter gets flat IDs
# - Heatmap gets TUPLE pairs
plotter.add_numeric_data(
    data_list=[umap_data, heatmap_data], 
    global_ids_list=[scatter_ids, id_pairs],  # id_pairs has tuples!
    titles=["Scatter View", "Heatmap (hover shows 2 images)"]
)

plotter.show(super_title="Heatmap with Tuple IDs Test")

In [8]:
# Clear the internal grid to free up references to large base64 strings
plotter._init_storage()
plotter.thumb_cache.clear()